# Sonified & Interactive Wind Map

In [ ]:
import ipytone
import ipywidgets
import numpy as np
import matplotlib.pyplot as plt

## Synthesizing the sound of wind

1. Create a noise source + a few other audio processing nodes to simulate the sound of wind

In [ ]:
noise = ipytone.Noise(volume=-5).start()
gain = ipytone.Gain()
env = ipytone.AmplitudeEnvelope(attack=1, release=2)
lp_filter = ipytone.Filter(frequency=1000, type="lowpass", rolloff=-24, q=4)
eq = ipytone.EQ3()
panner3d = ipytone.Panner3D()
delay = ipytone.PingPongDelay(wet=0.2)
dist = ipytone.Distortion(wet=0.01)
reverb = ipytone.Reverb(wet=0.2)

noise.chain(
    env,
    gain,
    lp_filter,
    eq,
    panner3d,
    delay,
    dist,
    reverb,
    ipytone.get_destination(),
);

In [ ]:
env.trigger_attack_release(3)

2. Add signal nodes to modulate the sound depending on wind speed

In [ ]:
wind_speed_sig = ipytone.Signal(value=0.2, min_value=0, max_value=1)

gain_scaler = ipytone.WaveShaper(curve=[0, 0.4, 1])
dist_scaler = ipytone.WaveShaper(curve=[0, 0.01, 0.1])
frequency_scaler = ipytone.WaveShaper(curve=[0, 500, 1000])
q_scaler = ipytone.WaveShaper(curve=[0, 2, 6])

wind_speed_sig.chain(gain_scaler, gain.gain)
wind_speed_sig.chain(dist_scaler, dist.wet)
wind_speed_sig.chain(frequency_scaler, lp_filter.frequency)
wind_speed_sig.chain(q_scaler, lp_filter.q)

3. Control wind speed using a slider widget

In [ ]:
wind_speed_slider = ipywidgets.FloatSlider(value=0.2, min=0, max=1, step=0.005)

l = ipywidgets.jslink((wind_speed_slider, "value"), (wind_speed_sig, "value"))

wind_speed_slider

In [ ]:
env.trigger_attack()

In [ ]:
env.trigger_release()

4. Tweak some parameter values to tune the wind sound

In [ ]:
delay.delay_time.value = 2.0
delay.feedback.value = 0.1

# more rumbling
eq.low_frequency.value = 200
eq.low.value = 7
eq.mid.value = -2.5
eq.high.value = 3

dist.distortion = 0.4

gain_scaler.curve = [0, 0.2, 2]
frequency_scaler.curve = [0, 100, 2000]
q_scaler.curve = [1, 4]

# move wind (noise) source above listener head
# (10m wind, and also for smoother x/z transitions)
panner3d.position_y.value = 3

noise.volume.value = 3

## Modulate the sound of wind with actual data

Using ERA5 monthly averaged northward and eastward components of wind at a height of 10m on a low resolution global grid (dataset downloaded from https://cluster.klima.uni-bremen.de/~fmaussion/teaching/climate/ERA5_LowRes_MonthlyAvg_uvslp.nc).

In [ ]:
import xarray as xr

ds = xr.open_dataset("assets/ERA5_LowRes_Monthly_uvslp.nc")

ds

In [ ]:
# wind components u, v -> wind speed and direction

ds["wind_speed"] = np.sqrt(ds.u10**2 + ds.v10**2)
ds["wind_speed_norm"] = ds.wind_speed / ds.wind_speed.max()

wind_angle = 1 * np.pi / 2 - np.arctan2(-ds.u10, -ds.v10)
ds["wind_pos_x"] = np.cos(wind_angle)
ds["wind_pos_z"] = np.sin(wind_angle)

In [ ]:
def make_wind(lat, lon, duration=50, speed_avg=5, speed_scale=2, pos_scale=2):
    """Generate wind sound from data at a given location."""

    ds_point = ds.sel(longitude=lon, latitude=lat, method="nearest")
    da_speed_avg = (
        ds_point.wind_speed_norm
        .rolling(time=speed_avg).mean()
        .dropna("time")
    )

    wind_speed_sig.cancel_scheduled_values(None)
    wind_speed_sig.set_value_curve_at_time(
        da_speed_avg.values * speed_scale,
        None,
        duration
    )

    panner3d.position_x.cancel_scheduled_values(None)
    panner3d.position_x.set_value_curve_at_time(
        ds_point.wind_pos_x.values * pos_scale,
        None,
        duration
    )

    panner3d.position_z.cancel_scheduled_values(None)
    panner3d.position_z.set_value_curve_at_time(
        ds_point.wind_pos_z.values * pos_scale,
        None,
        duration
    )

In [ ]:
env.trigger_attack()

In [ ]:
make_wind(44, 5)

In [ ]:
env.trigger_release()

## Interactive map

The time averaged (full time period) wind is shown as a global and iteractive velocity map, while the actual wind speed and direction is audio-generated at a given location (draggable marker) on the map.

In [ ]:
import ipyleaflet
from ipyleaflet.velocity import Velocity

In [ ]:
speed_widget = ipywidgets.FloatSlider(
    value=0, min=0, max=1.2, description="Speed", disabled=True, readout=False
)
speed_widget.style.handle_color = "blue"
pos_x_widget = ipywidgets.FloatSlider(
    value=0, min=-2.2, max=2.2, description="W / E", disabled=True, readout=False
)
pos_x_widget.style.handle_color = "blue"
pos_z_widget = ipywidgets.FloatSlider(
    value=0, min=-2.2, max=2.2, description="S / N", disabled=True, readout=False
)
pos_z_widget.style.handle_color = "blue"

ls = wind_speed_sig.schedule_jsdlink((speed_widget, "value"))
lx = panner3d.position_x.schedule_jsdlink((pos_x_widget, "value"))
lz = panner3d.position_z.schedule_jsdlink((pos_z_widget, "value"))

In [ ]:
paris_latlon = (48.85, 2.35)

m = ipyleaflet.Map(
    zoom=2,
    center=paris_latlon,
    basemap=ipyleaflet.basemaps.Esri.WorldImagery,
    layout=ipywidgets.Layout(height="600px")
)

wind = Velocity(
    data=ds.mean("time"),
    zonal_speed="u10",
    meridional_speed="v10",
    latitude_dimension="latitude",
    longitude_dimension="longitude",
    velocity_scale=0.05,
    max_velocity=20
)
m.add_layer(wind)

wind_control = ipyleaflet.WidgetControl(
    widget=ipywidgets.VBox([speed_widget, pos_x_widget, pos_z_widget]),
    position="topright"
)
m.add_control(wind_control)

icon = ipyleaflet.AwesomeIcon(
    name="headphones",
    marker_color='red',
    icon_color='white',
)

marker = ipyleaflet.Marker(location=paris_latlon, icon=icon)
m.add_layer(marker)

def on_location_changed(event):
    make_wind(*event["new"])

marker.observe(on_location_changed, "location")

In [ ]:
env.trigger_attack()
make_wind(*paris_latlon)
m

## Dispose and close (audio) widgets

In [ ]:
wind_speed_sig.cancel_scheduled_values(None)
panner3d.position_x.cancel_scheduled_values(None)
panner3d.position_z.cancel_scheduled_values(None)

env.trigger_release()

In [ ]:
m.close()

In [ ]:
l.unlink()
ls.unlink()
lx.unlink()
lz.unlink()

In [ ]:
noise.dispose()
env.dispose()
gain.dispose()
lp_filter.dispose()
eq.dispose()
panner3d.dispose()
delay.dispose()
dist.dispose()
reverb.dispose()

wind_speed_sig.dispose()
frequency_scaler.dispose()
q_scaler.dispose();